In [12]:
# importing libraries

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, ZeroPadding2D, GlobalAveragePooling2D

In [5]:
# Hyper parameters
BATCH_SIZE = 32
IMG_WIDTH, IMG_HEIGHT = 64, 64

TRAIN_DATA_PATH = './data/train/'
TEST_DATA_PATH = './data/test'
VALID_DATA_PATH = './data/valid'


In [6]:
def get_generator_from_dir(directory_path, batch_size_):
    data_gen = ImageDataGenerator(rescale=1./255)
    data_batch_generator = data_gen.flow_from_directory(
        directory_path, color_mode = 'rgb', target_size = (IMG_WIDTH, IMG_HEIGHT), 
        batch_size=batch_size_)
    
    return data_batch_generator

In [7]:
train_data = get_generator_from_dir(TRAIN_DATA_PATH, BATCH_SIZE)
validation_data = get_generator_from_dir(VALID_DATA_PATH, BATCH_SIZE)
test_data = get_generator_from_dir(TEST_DATA_PATH, BATCH_SIZE)

Found 4503 images belonging to 2 classes.
Found 333 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [17]:
activation_layer = tf.keras.layers.LeakyReLU(alpha=0.1)
visible = Input(shape=(IMG_WIDTH,IMG_HEIGHT,3),name='ip')
conv1 = Conv2D(128, kernel_size=3, strides=(1,1), padding="valid", activation=activation_layer,name='cn1')(visible)
pool1 = MaxPooling2D(pool_size=(3, 3),strides=(2,2), padding="valid",name='pl1')(conv1)

conv2 = Conv2D(64, kernel_size=5, strides=(2,2), padding="valid", activation=activation_layer,name='cn2')(pool1)
pool2 = MaxPooling2D(pool_size=(3, 3),strides=(2,2), padding="valid",name='pl2')(conv2)

flat = Flatten(name='flat')(pool2)
hidden1 = Dense(32, activation=activation_layer,name='hd1')(flat)
output = Dense(2, activation='softmax',name='op')(hidden1)
model = Model(inputs=visible, outputs=output)

# Optimizers, loss function and performance metrics
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
cat_loss = tf.keras.losses.CategoricalCrossentropy()
acc_metrics = [tf.keras.metrics.CategoricalAccuracy()]

# Compiling the model
model.compile(optimizer=opt, loss=cat_loss, metrics=acc_metrics)

In [18]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ip (InputLayer)             [(None, 64, 64, 3)]       0         
                                                                 
 cn1 (Conv2D)                (None, 62, 62, 128)       3584      
                                                                 
 pl1 (MaxPooling2D)          (None, 30, 30, 128)       0         
                                                                 
 cn2 (Conv2D)                (None, 13, 13, 64)        204864    
                                                                 
 pl2 (MaxPooling2D)          (None, 6, 6, 64)          0         
                                                                 
 flat (Flatten)              (None, 2304)              0         
                                                                 
 hd1 (Dense)                 (None, 32)                7376

In [19]:
model_train = model.fit(train_data, batch_size = BATCH_SIZE, validation_data = validation_data, epochs=10)

Epoch 1/10
141/141 [==============================] - 165s 1s/step - loss: 0.6006 - categorical_accuracy: 0.6755 - val_loss: 0.5691 - val_categorical_accuracy: 0.6577
Epoch 2/10
141/141 [==============================] - 170s 1s/step - loss: 0.5149 - categorical_accuracy: 0.7304 - val_loss: 0.4994 - val_categorical_accuracy: 0.7267
Epoch 3/10
141/141 [==============================] - 195s 1s/step - loss: 0.4862 - categorical_accuracy: 0.7524 - val_loss: 0.4817 - val_categorical_accuracy: 0.7598
Epoch 4/10
141/141 [==============================] - 206s 1s/step - loss: 0.4646 - categorical_accuracy: 0.7650 - val_loss: 0.4563 - val_categorical_accuracy: 0.7688
Epoch 5/10
141/141 [==============================] - 86s 607ms/step - loss: 0.4350 - categorical_accuracy: 0.7897 - val_loss: 0.4525 - val_categorical_accuracy: 0.7628
Epoch 6/10
141/141 [==============================] - 65s 464ms/step - loss: 0.3995 - categorical_accuracy: 0.8159 - val_loss: 0.3876 - val_categorical_accuracy: 0

In [20]:
y = model.evaluate(test_data)

16/16 [==============================] - 2s 134ms/step - loss: 0.3106 - categorical_accuracy: 0.8800


In [21]:
model.save('cdd_model.h5')